## Introduction
**Graph** is a data structure consisting of **nodes**, **edges** that connect nodes, and potentially additional data attached to nodes and/or edges.


Typical examples:
* social networks
* biological networks
* transportation networks
* computer networks
* ontologies
* knowledge graphs


Main graph types:
* by the presence of edge direction:
    * undirected graphs: the two nodes of an edge are "equal"; there is no notion of edge "from a to b" and "from b to a"
    * directed graphs (or digraphs): the edge is going "from" one node "to" another
* by the allowed number of edges between nodes:
    * simple graphs: each pair of nodes can have at most one edge
    * multigraphs: there can be more than one edge
* by the presence of edges connecting node with itself (**loops**):
    * with loops
    * without loops
* by the presence of weights:
    * weighted: there are numbers attached to edges
    * unweighted
* by the number of nodes:
    * finite: there is a finite number of nodes
    * infinite: there are infinitely many nodes
* by the possibility of changing over time:
    * dynamic: if nodes/edges can be added/removed over time
    * static: if the graph is given as is and not allowed to be changed


Here I consider only *finite* graphs.

## Mathematical definitions

A **simple undirected graph** is a pair $(V, E)$ where
* $V$ is a set of **nodes** (also called **vertices**)
* $E$ is a set of **edges** (also called **links**)
    * edges are represented as *unordered* pairs (or sets) of vertices: edge $\{a, b\}$ is the same as edge $\{b, a\}$
    
For **simple directed graph** $E$ contains of *ordered* pairs of vertices: $(a, b)$ and $(b, a)$ are two differente edges. There can be both of them, one of them, or none of them between any pair of nodes.
    
For **(un)directed multigraph** $E$ is a *multiset* instead of *set*, so the same pair can be present more than once.


A **weighted graph** is a triple $(V, E, w)$ where $w: E \rightarrow \mathbb{R}$ is a function that assigns **weight** $w(e)$ to each edge $e \in E$.


The **order** of a graph is the number of its vertices $|V|$.

The **size** of a graph is the number of its edges $E$.


### Adjacency matrix

Two nodes $v'$ and $v''$ are called **adjacent** if there is an edge from $v'$ to $v''$.

Since for undirected graph, there is no notion of edge direction, the adjacency relation is symmetric for them: $v'$ is adjacent to $v''$ if and only if $v''$ is adjacent to $v'$. This is not necessarily the case for directed graphs.

An **adjacency matrix** of a simple graph $G$ is a matrix $A$ of size $|V| \times |V|$ such that
$$ A_{ij} =
\begin{cases}
1 & \text{there is an edge from {i}-th to {j}-th node} \\
0 & \text{otherwise}
\end{cases} $$

For multigraphs,
$$ A_{ij} = \text{the number of edges from {i}-th to {j}-th node} $$

For simple weighted graphs,
$$ A_{ij} =
\begin{cases}
1 & \text{the weight from an edge from {i}-th to {j}-th node} \\
0 & \text{if there is no edge}
\end{cases} $$

I couldn't find or come up with any meaningful definition of an adjacence matrix for weighted multigraphs.

The adjacency matrices of undirected graphs are always symmetric.

The adjacency matrices of simple graphs always have zeros on the main diagonal.


### Density of a simple graph
**Graph density** of a simple undirected graph without loops is the ratio between the number of edges it actually has and the number of edges it *can* have:
$$D(G) = \frac{|E|}{\frac{|V| * (|V| - 1)}{2}} = \frac{2 |E|}{|V| (|V| - 1)}$$

For a simple undirected graph that can have loops,
$$D(G) = \frac{2 |E|}{|V| (|V| + 1)}$$

For a simple directed graph without loops,
$$D(G) = \frac{|E|}{|V| (|V| - 1)}$$

For a simple directed graph that can have loops,
$$D(G) = \frac{|E|}{|V|^2}$$

## Graph representations

These are the three main graph representations and their *potential* pros and cons (potential because they depend on the use case):
* **adjacency matrix**:
    * pro: trivial to check if two nodes are connected: $O(1)$ complexity
    * pro: can be efficiently represented as a sparse matrix for graphs with low density
    * con: adding or removing edges is costly: matrix has to be resized
    * con: space complexity for large highly dense graphs is $O(|V|^2)$
    * con: not applicable for weighted multigraphs
* **edge list**: literally a list of edges
    * pro: applicable for any graph type defined above
    * pro: space complexity $O(|E|)$ for any graph
    * con: hard to check if two vertices are connected (need to traverse over full list in the worst case)
* **adjacency list**: a mapping from a node $v$ to the list of its adjacent nodes (such that there is an edge from $v$ to these nodes)
    * pro: space complexity $O(|V| + |E|)$ which strikes a good balance for not too-dense and not too-sparse graphs
    * pro: the most efficient way to extract node's adjacent neighbors which is important for many algorithms
    * con: checking whether node $u$ is adjacent to node $v$ requries iteration over full list of node $v$
    * con: for directed graphs, checking whether node $v$ is adjacent to node $u$ or getting all nodes that $v$ is adjacent to requires iteration over full representation OR storing two lists per node which doubles the memory consumption and slightly complicates the data structure

If the graph is highly dense, then each of these methods has to store all edges, so the space complexity is $O(|V|^2)$ in the worst case.

In [1]:
from pprint import pprint 

import networkx as nx
import numpy as np
import scipy

## Creating simple undirected graphs in NetworkX

In [2]:
V = [0, 1, 2, 3, 4, 5]
E = [(0, 1), (1, 2), (1, 3), (3, 4)]

In [3]:
# Create simple undirected graph from ane edge list
# default constructor is nx.Graph which is for undirected graphs
G1 = nx.from_edgelist(E)
# Note that node 5 is absent since it's not found in the edge list
print('V1 =', G1.nodes)
print('|V1| =', G1.number_of_nodes())
print('E1 =', G1.edges)
print('|E1| =', G1.number_of_edges())

V1 = [0, 1, 2, 3, 4]
|V1| = 5
E1 = [(0, 1), (1, 2), (1, 3), (3, 4)]
|E1| = 4


In [4]:
nx.to_edgelist(G1)

EdgeDataView([(0, 1, {}), (1, 2, {}), (1, 3, {}), (3, 4, {})])

In [5]:
nx.to_pandas_edgelist(G1, source='from', target='to')

,from,to
0,0,1
1,1,2
2,1,3
3,3,4


In [6]:
nx.to_pandas_adjacency(G1)

,0,1,2,3,4
0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0


In [7]:
A1_scipy = nx.adjacency_matrix(G1)  # This is sparse matrix
print('type(A1_scipy) =', type(A1_scipy))
print('A1_scipy.shape =', A1_scipy.shape)
print(A1_scipy)
print()

print('# Converting A1_scipy to numpy.NDarray')
A1_ndarray = A1_scipy.toarray()
print('type(A1_ndarray) =', type(A1_ndarray))
print('A1_ndarray is symmetric:', scipy.linalg.issymmetric(A1_ndarray))
print(A1_ndarray)
print()

A1 = nx.to_numpy_array(G1)
print('type(A1) =', type(A1))
print('A1.shape =', A1.shape)
print('A1 == A1_ndarray:', (A1 == A1_ndarray).all())

type(A1_scipy) = <class 'scipy.sparse._arrays.csr_array'>
A1_scipy.shape = (5, 5)
  (0, 1)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (2, 1)	1
  (3, 1)	1
  (3, 4)	1
  (4, 3)	1

# Converting A1_scipy to numpy.NDarray
type(A1_ndarray) = <class 'numpy.ndarray'>
A1_ndarray is symmetric: True
[[0 1 0 0 0]
 [1 0 1 1 0]
 [0 1 0 0 0]
 [0 1 0 0 1]
 [0 0 0 1 0]]

type(A1) = <class 'numpy.ndarray'>
A1.shape = (5, 5)
A1 == A1_ndarray: True


## Creating simple digraphs in NetworkX

In [9]:
V = [0, 1, 2, 3, 4, 5]
E = [(0, 1), (1, 2), (1, 3), (3, 4)]

G2 = nx.DiGraph()
G2.add_edges_from(E)
G2.add_nodes_from(V)

# Note that node 5 is present since it was explicitly added in V list
print('V2 =', G2.nodes)
print('|V2| =', G2.number_of_nodes())
print('E2 =', G2.edges)
print('|E2| =', G2.number_of_edges())

V2 = [0, 1, 2, 3, 4, 5]
|V2| = 6
E2 = [(0, 1), (1, 2), (1, 3), (3, 4)]
|E2| = 4


In [10]:
A2 = nx.to_numpy_array(G2)
print('A2.shape =', A2.shape)
print('A2 is symmetric:', scipy.linalg.issymmetric(A2))
print(A2)

A2.shape = (6, 6)
A2 is symmetric: False
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [11]:
# Creating graph from an adjacency matrix
G3 = nx.from_numpy_array(A2, create_using=nx.DiGraph)
print('G3.nodes == G2.nodes:', G3.nodes == G2.nodes)
print('G3.edges == G2.edges:', G3.edges == G2.edges)
A3 = nx.adjacency_matrix(G3).toarray()
print('A3 == A2:', (A3 == A2).all())

G3.nodes == G2.nodes: True
G3.edges == G2.edges: True
A3 == A2: True


In [12]:
# Creating graph from an adjacency mapping 
adjacency_mapping = {
    0: [1],
    1: [2, 3],
    2: [],
    3: [4],
    4: [],
    5: []
}
G4 = nx.from_dict_of_lists(adjacency_mapping, create_using=nx.DiGraph)
print('G4.nodes == G2.nodes:', G4.nodes == G2.nodes)
print('G4.edges == G2.edges:', G4.edges == G2.edges)
A4 = nx.to_numpy_array(G4)
print('A4 == A2:', (A4 == A2).all())

G4.nodes == G2.nodes: True
G4.edges == G2.edges: True
A4 == A2: True


In [13]:
# Converting graph to adjacency mapping
for key, val in nx.to_dict_of_lists(G4).items():
    print(f'{key}: {val}')

0: [1]
1: [2, 3]
2: []
3: [4]
4: []
5: []


In [14]:
# Converting graph to adjacency mapping -- another version
for key, val in nx.to_dict_of_dicts(G4).items():
    print(f'{key}: {val}')

0: {1: {}}
1: {2: {}, 3: {}}
2: {}
3: {4: {}}
4: {}
5: {}


## Creating multi(di)graphs

In [15]:
G5 = nx.from_numpy_array(
    np.array([[1, 2, 0], [0, 0, 0], [0, 3, 1]]),
    parallel_edges=True,
    create_using=nx.MultiDiGraph)
print('V5 =', G5.nodes)
print('|V5| =', G5.number_of_nodes())
print('E5 =', G5.edges)
print('|E5| =', G5.number_of_edges())
print(nx.to_numpy_array(G5))

V5 = [0, 1, 2]
|V5| = 3
E5 = [(0, 0, 0), (0, 1, 0), (0, 1, 1), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0)]
|E5| = 7
[[1. 2. 0.]
 [0. 0. 0.]
 [0. 3. 1.]]


In [16]:
nx.to_pandas_edgelist(G5, edge_key='edge_key')

,source,target,edge_key,weight
0,0,0,0,1
1,0,1,0,1
2,0,1,1,1
3,2,1,0,1
4,2,1,1,1
5,2,1,2,1
6,2,2,0,1


In [17]:
nx.to_edgelist(G5)

OutMultiEdgeDataView([(0, 0, {'weight': 1}), (0, 1, {'weight': 1}), (0, 1, {'weight': 1}), (2, 1, {'weight': 1}), (2, 1, {'weight': 1}), (2, 1, {'weight': 1}), (2, 2, {'weight': 1})])

In [18]:
nx.to_dict_of_lists(G5)  # No multi-edges

{0: [0, 1], 1: [], 2: [1, 2]}

In [19]:
nx.to_dict_of_dicts(G5)

{0: {0: {0: {'weight': 1}}, 1: {0: {'weight': 1}, 1: {'weight': 1}}},
 1: {},
 2: {1: {0: {'weight': 1}, 1: {'weight': 1}, 2: {'weight': 1}},
  2: {0: {'weight': 1}}}}

## Attributes and weights

In [20]:
# Graph attributes can be added during creation
g = nx.DiGraph(name='Nice graph', label=42, type='digraph')
print('Original name:', g.name)
g.name = 'Awesome graph'
print('Name after change:', g.name)
print('Directed:', nx.is_directed(g))
print('Weighed:', nx.is_weighted(g))

Original name: Nice graph
Name after change: Awesome graph
Directed: True
Weighed: False


In [21]:
# Node attributes can be added with add_something commands:
g.add_node('a', ix=0, type='even')
g.add_nodes_from([
    ['b', dict(ix=1, type='odd', subtype='mono')],
    ['c', dict(ix=2, type='even', subtype='duo')],
    ['d', dict(ix=3, type='even')]
])
for v in g.nodes:
    ix = g.nodes[v]['ix']
    type_ = g.nodes[v]['type']
    try:
        subtype = g.nodes[v]['subtype']
    except KeyError:
        subtype = None
    print(f'node {v}; ix={ix}; type={type_}; subtype={subtype}')

node a; ix=0; type=even; subtype=None
node b; ix=1; type=odd; subtype=mono
node c; ix=2; type=even; subtype=duo
node d; ix=3; type=even; subtype=None


In [22]:
type(g.nodes)

networkx.classes.reportviews.NodeView

In [23]:
v = next(iter(g.nodes))
print(repr(v), type(v))

'a' <class 'str'>


In [24]:
type(g.nodes[v])

dict

In [25]:
g.nodes[v]

{'ix': 0, 'type': 'even'}

In [26]:
# Edge attributes can also be added with add_something commands:
g.add_edge('a', 'b', type='a-b', weight=1)
g.add_edge('a', 'c', weight=42)
g.add_edges_from([
    ['b', 'c', dict(type='b-c')],
    ['c', 'd', dict(type='c-d', weight=1)]
])
print('E:', g.edges)
for e in g.edges:
    type_ = g.edges[e].get('type')
    weight = g.edges[e].get('weight')
    print(f'edge {e}; type={type_}; weight={weight}')

E: [('a', 'b'), ('a', 'c'), ('b', 'c'), ('c', 'd')]
edge ('a', 'b'); type=a-b; weight=1
edge ('a', 'c'); type=None; weight=42
edge ('b', 'c'); type=b-c; weight=None
edge ('c', 'd'); type=c-d; weight=1


In [27]:
nx.is_weighted(g)

False

In [28]:
type(g.edges)

networkx.classes.reportviews.OutEdgeView

In [29]:
e = next(iter(g.edges))
print(repr(e), type(e))

('a', 'b') <class 'tuple'>


In [30]:
type(g['a'])

networkx.classes.coreviews.AtlasView

In [31]:
g['a']

AtlasView({'b': {'type': 'a-b', 'weight': 1}, 'c': {'weight': 42}})

In [32]:
type(g['a']['b'])

dict

In [33]:
g['a']['b']

{'type': 'a-b', 'weight': 1}

In [34]:
type(g.edges[e])

dict

In [35]:
g.edges[e]

{'type': 'a-b', 'weight': 1}

In [36]:
g['a']['b'] == g.edges[('a', 'b')]

True

In [37]:
nx.to_edgelist(g)

OutEdgeDataView([('a', 'b', {'type': 'a-b', 'weight': 1}), ('a', 'c', {'weight': 42}), ('b', 'c', {'type': 'b-c'}), ('c', 'd', {'type': 'c-d', 'weight': 1})])

In [38]:
nx.to_numpy_array(g)

array([[ 0.,  1., 42.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.]])

In [39]:
nx.to_pandas_adjacency(g)

,a,b,c,d
a,0.0,1.0,42.0,0.0
b,0.0,0.0,1.0,0.0
c,0.0,0.0,0.0,1.0
d,0.0,0.0,0.0,0.0


In [40]:
nx.to_pandas_edgelist(g)

,source,target,type,weight
0,a,b,a-b,1.0
1,a,c,NaN,42.0
2,b,c,b-c,NaN
3,c,d,c-d,1.0


## Neighbors

**Neighbors** of node $v$ are nodes that have an edge to or from $v$.

**Ego graph** (or **neighborhood graph**) of node $v$ is a graph that consists of node $v$, its neighbors and all the edges between these nodes.

The **degree** of node $v$ in an undirected graph is the number of edges to/from node $v$.

If the graph is simple, there can be at most one edge between two nodes, so the degree in this case is the number of neighbors.

In directed graph there are two notions of degree:
* **in-degree**: the number of edges going o $v$
* **out-degree**: the number of edges going from $v$

All three degree definitions above are applicable for simple graphs and for multigraphs as well.

In [41]:
# Ego graph inside undirected graph
G1_ego_1 = nx.ego_graph(G1, 1)
print('Nodes:', G1_ego_1.nodes)
print('Edges:', G1_ego_1.edges)

Nodes: [0, 1, 2, 3]
Edges: [(0, 1), (1, 2), (1, 3)]


In [42]:
# Ego graph inside directed graph
G3_ego_1 = nx.ego_graph(G3, 1)
print('Nodes:', G3_ego_1.nodes)
print('Edges:', G3_ego_1.edges)

Nodes: [1, 2, 3]
Edges: [(1, 2), (1, 3)]


In [43]:
# Degrees in undirected graph
print(G1.edges)
for v in G1.nodes:
    print(f'deg({v}) = {G1.degree(v)}')

[(0, 1), (1, 2), (1, 3), (3, 4)]
deg(0) = 1
deg(1) = 3
deg(2) = 1
deg(3) = 2
deg(4) = 1


In [44]:
# Degrees in directed graph
print(G2.edges)
for v in G2.nodes:
    print(f'deg-({v}) = {G2.in_degree(v)}')
    print(f'deg+({v}) = {G2.out_degree(v)}')
    print()

[(0, 1), (1, 2), (1, 3), (3, 4)]
deg-(0) = 0
deg+(0) = 1

deg-(1) = 1
deg+(1) = 2

deg-(2) = 1
deg+(2) = 0

deg-(3) = 1
deg+(3) = 1

deg-(4) = 1
deg+(4) = 0

deg-(5) = 0
deg+(5) = 0

